In [5]:
pip install boto3==1.34.66

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 81.0 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.32
    Uninstalling botocore-1.34.32:
      Successfully uninstalled botocore-1.34.32
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.32
    Uninstalling boto3-1.34.32:
      Successfully uninstalled boto3-1.34.32
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.32 requires botocore==1.34.32, but you have botocore 1.34.67 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
boto3.__version__

'1.34.66'

In [8]:
import boto3
import json
import base64
import time
import re
import os
from botocore.exceptions import ClientError
import logging
import redis
from datetime import datetime , date
from insights import construct_call_conversation , enrollment_prompt_generator , load_claude2 , sns_data_postprocessing  
from insights import sns_publisher , get_insights_prompt , get_enrollment_prompt , insights_prompt_generator

today_date = date.today()

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name= os.environ.get("Region") #"us-east-1",
)
 
bedrock = boto3.client(
    service_name='bedrock', 
    region_name= os.environ.get("Region")  #'us-east-1'
)
 
bedrock_agent_runtime = boto3.client(
    service_name="bedrock-agent-runtime",
    region_name= os.environ.get("Region") #"us-east-1",
)
# Initialize Redis client

redis_host = 'ch-agent-assist-redis-cluster-disabled.tjyhst.ng.0001.use1.cache.amazonaws.com'
#os.environ.get("RedisReadUrl") # 'ch-agent-assist-redis-cluster-disabled-ro.tjyhst.ng.0001.use1.cache.amazonaws.com'

redis_port = 6379 # Default Redis port is 6379

redis_password = None  # None if no password

redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)
 
prompt_bucket = 'ch-agent-assist-prompt-library-bucket'

file_key = 'prompts_library.json'

#AWS SDK for importing the KB
def retrieveAndGenerate(input, kbId):
    return bedrock_agent_runtime.retrieve_and_generate(
        input={
            'text': input
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': 'WWGXWYHCZE',
                'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-v2'
                }
            }
        )

def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_runtime.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults
            }
        }
    )

kb_response = retrieve("What is the average time to fill for the specialty pharmacy as CVS", "WWGXWYHCZE")["retrievalResults"]
print(kb_response)
#kb_response_payer_pharmacy_list = retrieve("What are the different type of insurance providers? and for each of insurance providers, provide the time to fill for all of the specialty pharamcies, provide the response in a structure JSON format", "WWGXWYHCZE")["retrievalResults"]
#print(kb_response_payer_pharmacy_list)
 
entity_dict_v1 = {
    "patientFirstName": "",
    "patientMiddleInitial": "",
    "patientLastName": "",
    "dob": "",
    "age": "",
    "gender": "",
    "preferredLanguage": "",
    "streetAddress": "",
    "streetName": "",
    "city": "",
    "state": "",
    "zipCode": "",
    "email": "",
    "phone": "",
    "phoneType": "",
    "priorTherapy": "",
    "diagnosisName": "",
    "diagnosisIcd10Code": "",
    "dateOfDiagnosis": "",
    "insuranceProvider": "",
    "insuranceId": "",
    "planType": "",
    "effectiveDate": "",
    "expiryDate": "",
    "recordType": "",
    "rxBin": "",
    "rxGroup": "",
    "rxPcn": "",
    "cardHolderRelationshipWithThePatient": "",
    "cardHolderName": "",
    "cardHolderDob": "",
    "prescriberName": "",
    "specialty": "",
    "address": "",
    "facilityName": "",
    "pharmacyName": "",
    "spPhone": "",
    "spFax": ""
}

entity_list = list(entity_dict_v1.keys())

entities = ""

for val in entity_list:
    entities += val + "," 

entities = entities[:len(entities)-1]
# Lambda handler to intgerate with AWS

def handler(event,context):
    start_timestamp = datetime.now()
    print(f'insights_data: handler: START: {datetime.now()}: {str(event)}')
    contact_id = json.loads(event["Records"][0]['body'])["streamConnectionId"] #event.get('conversation_id')
    #print(key_to_read)
    # Check if the key exists and read the value from Redis
    if contact_id and redis_client.exists(contact_id):
        value = redis_client.get(contact_id)
        print(value) #.split("contactId"))
    else:
        print('Key not found')
    #final_transcript = ""
    #for i in range(len(event['Records'])):
     #   final_transcript += "\n" + data_preprocessing(event['Records'][i])
    final_transcript = construct_call_conversation(value)
    #print(final_transcript)
    
    #sample_transcript = "Customer : Hello , I'm Mark."
    #prompt_enrollment = get_enrollment_prompt(prompt_bucket,file_key,'entity_extraction','enrollment_form_claude',final_transcript)
    prompt_enrollment = enrollment_prompt_generator(final_transcript,entities,today_date)
    #prompt_enrollment = enrollment_prompt_generator(sample_transcript,entities)
    entity_extraction_model_invoke_timestamp = datetime.now()
    enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
    entity_extraction_model_invoke_after_timestamp = datetime.now()
    print(enrollment_data)
    #json_data = data_postprocessing(enrollment_data)
    json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
    match = json_pattern.search(enrollment_data)
    extracted_json = match.group(0)
    enrollment_json_object = json.loads(extracted_json)
    print("Printing all the keys",list(enrollment_json_object.keys()))
    enrollment_keys = list(enrollment_json_object.keys())
    if "insuranceProvider" in enrollment_json_object.keys() and "pharmacyName" in enrollment_json_object.keys() and enrollment_json_object["insuranceProvider"] != "" and enrollment_json_object["pharmacyName"] != "":
        redis_key = str(json.loads(event["Records"][0]['body'])["streamConnectionId"]) + '::insights_data'
        if redis_client.exists(redis_key):
            print(f'redis_key:{redis_key}')
            insights_redis_data_value = redis_client.get(redis_key)
            print(f'insights_redis_data_value:{insights_redis_data_value}')
            insights_redis_data = eval(insights_redis_data_value)
            print(f'insights_redis_data : {insights_redis_data}')
        #redis_client.set(redis_key,str([{"insuranceProvider":'United Health','pharmacyName':'CVS','insights':'Insights'}]))
        #redis_client.set(redis_key,str([]))
        #insights_redis_data_value = redis_client.get(redis_key)
        #print(f'insights_redis_data_value:{insights_redis_data_value}')
        #insights_redis_data = eval(insights_redis_data_value)
        #print(f'insights_redis_data : {insights_redis_data}')
        if len(insights_redis_data) == 0:
            insurance_provider = enrollment_json_object["insuranceProvider"]
            pharmacyName = enrollment_json_object["pharmacyName"]
            #insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
            insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
            insights_model_invoke_timestamp = datetime.now()
            insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
            insights_model_invoke_after_timestamp = datetime.now()
            print("Generating Insights successfully", insights)
            sns_data = sns_data_postprocessing(event,insights)
            print("sns_data : ",sns_data)
            sns_publisher(sns_data)
            insights_redis_data.append({'insuranceProvider':enrollment_json_object['insuranceProvider'],'pharmacyName':enrollment_json_object['pharmacyName'],'insights':insights})
            print(f'new_value : {insights_redis_data}')
            redis_client.set(redis_key,str(insights_redis_data))
            return {'statusCode':200,'body':'Data Sent to SNS Sucessfully.'}
          
        for val in insights_redis_data:
            if val['insuranceProvider'] == enrollment_json_object['insuranceProvider'] and val['pharmacyName'] == enrollment_json_object['pharmacyName']:
                print(f'insights_redis_data:{insights_redis_data}')
                return {"statusCode" : 200 , "body" : "Generated Insight already present in the database."}
                print('Generated Insight already present in the database.')
            #pass
            else:
                print('Have come to else...')
                insurance_provider = enrollment_json_object["insuranceProvider"]
                pharmacyName = enrollment_json_object["pharmacyName"]
                #insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
                insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
                insights_model_invoke_timestamp = datetime.now()
                insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
                insights_model_invoke_after_timestamp = datetime.now()
                print("Generating Insights successfully", insights)
                sns_data = sns_data_postprocessing(event,insights)
                print("sns_data : ",sns_data)
                sns_publisher(sns_data)
                insights_redis_data.append({'insuranceProvider':enrollment_json_object['insuranceProvider'],'pharmacyName':enrollment_json_object['pharmacyName'],'insights':insights})
                print(f'insights_redis_data:{insights_redis_data}')
                redis_client.set(redis_key,str(insights_redis_data))
                end_timestamp = datetime.now()
                duration_till_entity_model_invoking = entity_extraction_model_invoke_timestamp - start_timestamp
                duration_of_response_of_entity_model = entity_extraction_model_invoke_after_timestamp - entity_extraction_model_invoke_timestamp
                duration_between_respone_of_entityModel_and_insightsModel = insights_model_invoke_timestamp - entity_extraction_model_invoke_after_timestamp 
                duration_of_response_of_insights_model = insights_model_invoke_after_timestamp - insights_model_invoke_timestamp
                duration_till_end = end_timestamp - insights_model_invoke_after_timestamp
                print(f'insights_data: handler: END: duration_start_to_entity_model::{duration_till_entity_model_invoking}:: duration_entity_model::{duration_of_response_of_entity_model}:: duration_entity_model_to_insights_model::{duration_between_respone_of_entityModel_and_insightsModel}:: duration_insights_model::{duration_of_response_of_insights_model}:: duration_insight_model_to_end::{duration_till_end}:: output: {json.dumps(sns_data)}')
                return {"statusCode": 200,"body": "Data Sent to SNS Sucessfully"}
        else:
            new_value_list = []
            redis_client.set(redis_key,str([])) 
            insurance_provider = enrollment_json_object["insuranceProvider"]
            pharmacyName = enrollment_json_object["pharmacyName"]
            insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
            insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
            insights_model_invoke_timestamp = datetime.now()
            insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
            insights_model_invoke_after_timestamp = datetime.now()
            print("Generating Insights successfully", insights)
            sns_data = sns_data_postprocessing(event,insights)
            print("sns_data : ",sns_data)
            sns_publisher(sns_data)
            new_value_list.append({'insuranceProvider':enrollment_json_object['insuranceProvider'],'pharmacyName':enrollment_json_object['pharmacyName'],'insights':insights})
            print(f'new_value : {new_value_list}')
            redis_client.set(redis_key,str(new_value_list))
            print('Insights Lambda Triggered.')
            end_timestamp = datetime.now()
            duration_till_entity_model_invoking = entity_extraction_model_invoke_timestamp - start_timestamp
            duration_of_response_of_entity_model = entity_extraction_model_invoke_after_timestamp - entity_extraction_model_invoke_timestamp
            duration_between_respone_of_entityModel_and_insightsModel = insights_model_invoke_timestamp - entity_extraction_model_invoke_after_timestamp 
            duration_of_response_of_insights_model = insights_model_invoke_after_timestamp - insights_model_invoke_timestamp
            duration_till_end = end_timestamp - insights_model_invoke_after_timestamp
            print(f'insights_data: handler: END: duration_start_to_entity_model::{duration_till_entity_model_invoking}:: duration_entity_model::{duration_of_response_of_entity_model}:: duration_entity_model_to_insights_model::{duration_between_respone_of_entityModel_and_insightsModel}:: duration_insights_model::{duration_of_response_of_insights_model}:: duration_insight_model_to_end::{duration_till_end}:: output: {json.dumps(sns_data)}')
            return {"statusCode": 200,"body": "Data Sent to SNS Sucessfully"}
    else:
        #sns_data = sns_data_postprocessing(event," ")
        print("No Insigts Generated.")
        return {"statusCode": 200,"body": "No Insigts Generated."}
    #SQS_Publisher(sqs_data)
    #print("sns_data : ",sns_data)

[{'content': {'text': 'Data Set 3  \tinsuranceType\tInsurance Provider\tPatient with\tSpecialty Pharmacy\tConversion %\tTime To Fill\t% of patients staying for more than a year\t% of Patient Requires PA\tTTF when PA required\tPA% Approved\t% of Patient on treatment if used with financial Assistance program\t\tInsights \tCOMMERCIAL\tEXPRESS SCRIPTS\tHub\tWalgreens\t74\t10\t79\t60\t12\t77\t86\t\tComparision and choices between the SPs \tCOMMERCIAL\tEXPRESS SCRIPTS\tHub\tCVS\t92\t15\t77\t67\t17\t78\t60\t\tHow the insurance type for the same insurance plan has an effect on the TTF \tCOMMERCIAL\tEXPRESS SCRIPTS\tHub\tElixir\t70\t11\t61\t53\t13\t87\t59 \tCOMMERCIAL\tEXPRESS SCRIPTS\tHub\tCigna\t81\t17\t62\t38\t19\t79\t80 \tCOMMERCIAL\tEXPRESS SCRIPTS\tSP\tWalgreens\t70\t12\t60\t79\t14\t82\t78 \tCOMMERCIAL\tEXPRESS SCRIPTS\tSP\tCVS\t89\t19\t67\t77\t21\t79\t62 \tCOMMERCIAL\tEXPRESS SCRIPTS\tSP\tElixir\t65\t12\t53\t61\t14\t85\t67 \tCOMMERCIAL\tEXPRESS SCRIPTS\tSP\tCigna\t80\t16\t38\t62\t18\t90\

In [9]:
print(kb_response[0]["content"]["text"])

Data Set 3  	insuranceType	Insurance Provider	Patient with	Specialty Pharmacy	Conversion %	Time To Fill	% of patients staying for more than a year	% of Patient Requires PA	TTF when PA required	PA% Approved	% of Patient on treatment if used with financial Assistance program		Insights 	COMMERCIAL	EXPRESS SCRIPTS	Hub	Walgreens	74	10	79	60	12	77	86		Comparision and choices between the SPs 	COMMERCIAL	EXPRESS SCRIPTS	Hub	CVS	92	15	77	67	17	78	60		How the insurance type for the same insurance plan has an effect on the TTF 	COMMERCIAL	EXPRESS SCRIPTS	Hub	Elixir	70	11	61	53	13	87	59 	COMMERCIAL	EXPRESS SCRIPTS	Hub	Cigna	81	17	62	38	19	79	80 	COMMERCIAL	EXPRESS SCRIPTS	SP	Walgreens	70	12	60	79	14	82	78 	COMMERCIAL	EXPRESS SCRIPTS	SP	CVS	89	19	67	77	21	79	62 	COMMERCIAL	EXPRESS SCRIPTS	SP	Elixir	65	12	53	61	14	85	67 	COMMERCIAL	EXPRESS SCRIPTS	SP	Cigna	80	16	38	62	18	90	69 	GOVERNMENT	EXPRESS SCRIPTS	Hub	Walgreens	68	10	77	45	12	81	45 	GOVERNMENT	EXPRESS SCRIPTS	Hub	CVS	87	14	48	50	16	86	58 	GOV

In [15]:
def insights_prompt_generator(kb_response):
    prompt_claude = """Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance details.Can you tell what all type of insurance types are there as present in the kb  \" """ + kb_response + """ \" 


Assistant:
"""
    return prompt_claude

In [41]:
#Please Provide the Insight in brief without missing any important details in about 1000 .

# Prompt Testing

In [16]:
print(kb_response)
insights_prompt = insights_prompt_generator(kb_response)
print(insights_prompt)

The different types of insurance mentioned in the search results are: - Commercial - Government - Medicaid - Medicare
Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance details.Can you tell what all type of insurance types are there as present in the kb  " The different types of insurance mentioned in the search results are: - Commercial - Government - Medicaid - Medicare " 


Assistant:



passing prompt directly to the kb, insights generated:  Based on the search results, for a patient with Humana insurance, CVS pharmacy has the lowest average time to fill prescription compared to other pharmacies like Walgreens, Elixir and Cigna. 

For example, in search result 1, for a Humana commercial plan patient using the hub model, the average time to fill at CVS is 9 days compared to 10 days at Walgreens, 17 days at Elixir, and 15 days at Cigna.

Therefore, I would recommend CVS pharmacy for this patient to get their prescription filled the fastest.


In [52]:
kb_response_payer_pharmacy_list = retrieveAndGenerate("What are the different insurance providers? provide the average time to fill for all of the specialty pharamcies for the insurance providers", "WWGXWYHCZE")["output"]["text"]
#print("Combination of payer with phramacies are:",kb_response_payer_pharmacy_list) 
#kb_response_payer_pharmacy = retrieveAndGenerate("Provide all the insurance provider and specialty pharmacies along with the Time to Fill for the extracted, provide the output in a structured JSON format", "WWGXWYHCZE")["output"]["text"]
#print(kb_response_payer_pharmacy) 

def insights_prompt_generator_payer_pharmacy(insurance_provider,pharmacy_name,kb_response_payer_pharmacy_list):
    prompt_claude_payer_pharmacy = """Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance and pharmacy details related insights like for example suggesting which pharmacy to select based on the
patient's Insurance provider and the pharmacy using metrics like how soon the other pharmacies if having a quicker dispense time can be helpful.
The lesser the number of days to dispense the medication,the higher are the chances of recommendation of that pharmacy.

The less is the average time to fill, lesser is the number of days to dispense the medication.

If \" """ + insurance_provider + """ \"  or \" """ + pharmacy_name + """ \" is not present in \" """ + kb_response_payer_pharmacy_list + """ \" , then just provide a response telling that you don't have sufficient information to provide any insights and do not follow any of the information in rest of this prompt.

The patients's insurance provider is  \" """ + insurance_provider + """ \" and pharmacy is  \" """ + pharmacy_name + """ \" and use the following insurance statistics data to provide the insights:
\" """ + kb_response_payer_pharmacy_list + """ \".


Only provide the response when you have the values for both insurance provider and pharmacy name from the patient.Do not generate any insights unless you have both these information.
Also do not recommend any insights if the patient already has the pharmacy which is the quickest according to the insurance statistics and just suggest the patient to continue
with their existing pharmacy and not change their choice of pharmacy.


Please Provide the recommendation in brief without missing any important details in about 6 lines in less than 1000 characters

Provide the response in a structured and easily readable format


Assistant:
"""
    return prompt_claude_payer_pharmacy



In [53]:
#kb_response_payer_pharmacy_list = retrieveAndGenerate("What are the different insurance providers? provide the average time to fill for all of the specialty pharamcies for the insurance providers", "WWGXWYHCZE")["output"]["text"]
print("Combination of payer with phramacies are:",kb_response_payer_pharmacy_list) 
sample_data = 'Customer : My insuranceProvider is Humana and my preferred Pharmacy is CVS'
prompt_enrollment = enrollment_prompt_generator(sample_data,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
print(insurance_provider)
pharmacyName = enrollment_json_object["pharmacyName"]
print(pharmacyName)
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt_payer_pharmacy = insights_prompt_generator_payer_pharmacy(insurance_provider,pharmacyName,kb_response_payer_pharmacy_list)
#insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt_payer_pharmacy,0.5,0.9,1)
print("Generated insights: ",insights)


Combination of payer with phramacies are: The insurance providers mentioned are:
- Express Scripts
- Humana 
- United Health
- OptumRx
- Cigna The average time to fill for the specialty pharmacies per insurance provider is:

Express Scripts:
- Walgreens: 13 days 
- CVS: 16 days
- Elixir: 15 days
- Cigna: 16 days

Humana: 
- Walgreens: 13 days
- CVS: 15 days 
- Elixir: 15 days
- Cigna: 14 days  

United Health:
- Walgreens: 11 days
- CVS: 15 days
- Elixir: 17 days 
- Cigna: 13 days

OptumRx:  
- Walgreens: 14 days
- CVS: 16 days
- Elixir: 16 days
- Cigna: 15 days 

Cigna:
- No specialty pharmacy time to fill data
Humana
CVS
Generated insights:   Based on the information provided, the patient's insurance provider is Humana and their pharmacy is CVS. According to the insurance statistics data, for Humana insurance, CVS has an average time to fill of 15 days. This is longer than the 13 days for Walgreens and same as the 15 days for Elixir. 

Since the patient already has their prescription

In [46]:
sample_data_1 = 'Customer : My Insurance is with Green Health and my preferred Pharmacy is CVS.'
prompt_enrollment = enrollment_prompt_generator(sample_data_1,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
pharmacyName = enrollment_json_object["pharmacyName"]
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
print(insights)

 Unfortunately I do not have sufficient information about the patient's insurance provider and pharmacy to provide meaningful insights for this case. Let's move our conversation forward and I'll be happy to assist if more details become available.


In [18]:
sample_data_2 = 'Customer : My Insurance is with ABC Health and my preferred Pharmacy is CVS.'
prompt_enrollment = enrollment_prompt_generator(sample_data_2,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
pharmacyName = enrollment_json_object["pharmacyName"]
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
print(insights)

 I do not have sufficient information in the provided insurance statistics data to give any meaningful insights for this patient. The patient's insurance provider "ABC Health" and pharmacy "CVS" are not present in the data. Without having the dispense time metrics for this specific insurance provider and pharmacy combination, I cannot make an informed recommendation. I suggest continuing the conversation with the patient to gather the required details before providing pharmacy recommendations.


In [1]:
sample_data_3 = 'Customer : My Insurance is with United Health and my preferred Pharmacy is CVS.'
prompt_enrollment = enrollment_prompt_generator(sample_data_3,entities,date.today())
enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
json_pattern = re.compile(r'{(.+?)}', re.DOTALL)
match = json_pattern.search(enrollment_data)
extracted_json = match.group(0)
enrollment_json_object = json.loads(extracted_json)
insurance_provider = enrollment_json_object["insuranceProvider"]
pharmacyName = enrollment_json_object["pharmacyName"]
#insights_prompt = get_insights_prompt(prompt_bucket,file_key,'insight_generation','insights_claude',insurance_provider,pharmacyName,insurance_statistics)
insights_prompt = insights_prompt_generator(insurance_provider,pharmacyName,insurance_statistics)
insights_model_invoke_timestamp = datetime.now()
insights = load_claude2(bedrock_runtime,insights_prompt,0.5,0.9,1)
print(insights)

NameError: name 'enrollment_prompt_generator' is not defined

# Passing prompt directly in Ret and Gen of Kb

In [20]:
insuranceProvider='Humana'
pharmacyName='CVS'

def insights_prompt_generator_payer_pharmacy(insurance_provider,pharmacy_name):
    prompt_claude_payer_pharmacy = """Human: 
    
You are Agent assist tracking the Patient and agent conversation and help the agent recommend meaningful insights based on the 
insurance and pharmacy details related insights like for example suggesting which pharmacy to select based on the
patient's Insurance provider and the pharmacy using metrics like how soon the other pharmacies if having a quicker dispense time can be helpful.
The lesser the number of days to dispense the medication,the higher are the chances of recommendation of that pharmacy.

The less is the average time to fill, lesser is the number of days to dispense the medication.

The patients's insurance provider is  Humana and pharmacy is  CVS.

Assistant:
"""
    return prompt_claude_payer_pharmacy

insights_prompt_payer_pharmacy = insights_prompt_generator_payer_pharmacy(insuranceProvider,pharmacyName)

insights_from_kb = retrieveAndGenerate(insights_prompt_payer_pharmacy, "WWGXWYHCZE")["output"]["text"]
print("passing prompt directly to the kb, insights generated: ",insights_from_kb)